### OpenStreetMap Data Case Study (Boston)
by Mohamad Zeini Jahromi

## Introduction
OpenStreetMap (OSM) is a collaborative project to create a free editable map of the world. The creation and growth of OSM has been motivated by restrictions on use or availability of map information across much of the world, and the advent of inexpensive portable satellite navigation devices.

## Objective
While very useful, OSM data can be quite messy at times. In this project, I will walk you through the cleaning process of the data, storing data in the CSV format and analyzing it via SQL queries. 

I live in a small college town and for the purpose of this study, I chose one of my favorite cities in united states, Boston. The XML file (414 MB) has been downloaded from [Map Zen website](https://mapzen.com/data/metro-extracts/metro/boston_massachusetts/).

## Data wrangling
The following steps have been taken in our data wrangling projects:

- Creating a smaller size (4 MB) sample file from the original XML file.
- Auditing the types of tags and attributes
- Deciding which tags needs to be edited
- Systematically checking for inconsistencies
- Editing the inconsistent values
- Saving the data in CSV format
- Converting the CSV format to SQL Database
- Analyzing the data using SQL queries 

## Types of tags and attributes
I used the iterative parsing to process the map file and find out not only what tags are there, but also how many, to get the
feeling on how much of which data I can expect to have in the map.
The following code returns a dictionary with the tag name as the key and number of times this tag can be encountered in 
the map as value.


In [105]:
import xml.etree.cElementTree as ET
import pprint

# original OSM file and its sample
OSM_FILE = "boston_massachusetts.osm"             
SAMPLE_FILE = "boston_massachusetts_sample.osm"

# This function counts the unique number of tags in the given file
def count_tags(filename):                         
    tags = {}
    context = ET.iterparse(filename)
    for event, elem in context:
        if elem.tag not in tags.keys():
            tags[elem.tag] = 1
        else:
            tags[elem.tag] +=1
    return tags

# Prints out the tags and their count numbers
#tags = count_tags(SAMPLE_FILE)
tags = count_tags(OSM_FILE)
for k, v in tags.items(): print k, ":", v


node : 1931442
nd : 2323539
bounds : 1
member : 10959
tag : 900810
relation : 1295
way : 309066
osm : 1


The results show that **osm** and **bounds** are the root elements and other tags are child. Moreover, the majority of the information are located in **node** and **way** tags. 
There are many different child tags in the file and since cleaning all of them will take a lot of time and for the purpose of this study, I decided to check the following tags attributes for consistency.
- addr:street
- addr:state
- addr:postcode

## Improving Street Names

First we audit the **OSM FILE** to find the unexpected street types with respect to the appropriate ones in **expected list**.
The following function identifies all the unique attributes in this tags.


In [96]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

# A regular expression to find the end word of address string which can includes "." 
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

# List of expected street types 
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", 'Circle','Highway','Center','Turnpike','Way']

# This function creates a list of all unexpected street types 
# which are not in the expected list
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

# This function checks if the address type is a street type
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

# The main function to audit the OSM file and returns the unexpected street types
# and their respective examples
def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

# Print out the examples of unexpected street types
#street_types = audit(SAMPLE_FILE)
street_types = audit(OSM_FILE)
print 'Number of street types =', len(street_types)
for street_type, ways in street_types.items()[0:5]: print street_type, ":", ways

Number of street types = 61
6 : set(['South Station, near Track 6'])
1302 : set(['Cambridge Street #1302'])
Elm : set(['Elm'])
St, : set(['Walnut St,'])
Winsor : set(['Winsor'])


Then we create a **mapping list** to reflect the changes needed to fix the unexpected street types to the appropriate ones in the **expected list**. I have added items to mappings only for the actual problems I found in this **OSM FILE**, not a generalized solution, since that may and will depend on the particular area I'm auditing.

Finally, the following code (**update_street** function) fixes the street names. The function takes a string with street name as an argument and return the fixed name. I have provided a simple test so that you see what exactly is expected.


In [111]:
# Dictionary of unexpected street types as keys and their appropriate ones as values
mapping_street = {"Ave": "Avenue","Ave.":"Avenue","Ct":"Court","Dr":"Drive","Ext":"Exit",
           "HIghway":"Highway","Hwy":"Highway","Pkwy":"Parkway","Pl":"Place","Rd":"Road",
           "ST":"Street","Sq.":"Square","St":"Street","St,":"Street","St.":"Street",
           "Street.":"Street","rd.":"Road","st":"Street","street":"Street"}

# This function update street names using the "mapping_street" dictionary
def update_street(name, mapping):
    name = name.split(" ")
    if name[-1] in mapping.keys():
        name[-1] = mapping[name[-1]]
    name = " ".join(name)
    return name

# Print out the examples of update function
for street_type, ways in street_types.items()[0:5]:
    for name in ways:
        better_name = update_street(name, mapping_street)
        print name, "=>", better_name
        

South Station, near Track 6 => South Station, near Track 6
Cambridge Street #1302 => Cambridge Street #1302
Elm => Elm
Walnut St, => Walnut Street
Winsor => Winsor


## Improving State Names

First we audit the **OSM FILE** to find all types of state names and their respective numbers.
The following function identifies all the unique attributes in this tags.

In [119]:
# This function creates a list of all types of states  
def audit_state_type(state_types, state_name):
    if state_name not in state_types:
        state_types[state_name] = 1
    else:
        state_types[state_name] += 1

# This function checks if the address type is a state type
def is_state_name(elem):
    return (elem.attrib['k'] == "addr:state")

# The main function to audit the OSM file and returns all types of states
# in the OSM file
def audit(osmfile):
    osm_file = open(osmfile, "r")
    state_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_state_name(tag):
                    audit_state_type(state_types, tag.attrib['v'])
    osm_file.close()
    return state_types

# Print out all types of states 
#st_types = audit(SAMPLE_FILE)
state_types = audit(OSM_FILE)
print 'Number of state types =',len(state_types)
for k, v in state_types.items(): print k, ":", v


Number of state types = 7
ma : 6
MA : 2029
MA- MASSACHUSETTS : 60
Massachusetts : 13
Ma : 6
WA : 1
MASSACHUSETTS : 1


There are 7 different state types in our data file and only 1 case has 'WA' as state type and the rest belong to 'MA'.
The following code will change all notations of Massachusetts state to 'MA'. The function takes a string with state name as an argument and return the fixed name. I have provided a simple test so that you see what exactly is expected.


In [16]:
# Dictionary of unexpected state types as keys and their appropriate ones as values
mapping_state = { "MA- MASSACHUSETTS": "MA",
            "MASSACHUSETTS": "MA",
            "Ma": "MA",
            "Massachusetts": "MA",
            "ma": "MA"
            }

# This function update state names using the "mapping_state" dictionary
def update_state(name, mapping):
    if name in mapping.keys():
        name = mapping[name]
    return name

# Print out the examples of update function
for state_type, num in state_types.iteritems():
    better_name = update_state(state_type, mapping_state)
    print state_type, "=>", better_name
    

ma => MA
MA => MA
MA- MASSACHUSETTS => MA
Massachusetts => MA
Ma => MA
WA => WA
MASSACHUSETTS => MA


## Improving ZIP Codes

First we audit the **OSM FILE** to find the all notations of zip codes in our data file and their respective numbers.
The following function identifies all the unique attributes in this tags.


In [120]:
# This function creates a list of all types of zipcode  
def audit_zipcode(zipcode_types, zipcode):
    if zipcode not in zipcode_types:
        zipcode_types[zipcode] = 1
    else:
        zipcode_types[zipcode] += 1

# This function checks if the address type is a zipcode type        
def is_zipcode(elem):
    return (elem.attrib['k'] == "addr:postcode")

# The main function to audit the OSM file and returns all types of zipcode
# in the OSM file
def audit(osmfile):
    osm_file = open(osmfile, "r")
    zipcode_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_zipcode(tag):
                    audit_zipcode(zipcode_types, tag.attrib['v'])
    osm_file.close()
    return zipcode_types

# Print out all types of zipcode along with their cout numbers 
#st_types = audit(SAMPLE_FILE)
zipcode_types = audit(OSM_FILE)
print 'Number of zipcode types =',len(zipcode_types)
for k, v in zipcode_types.items()[0:5]: print k, ":", v


Number of zipcode types = 123
0239 : 1
02186 : 9
02184 : 3
02134-1327 : 1
02189 : 1


After auditing the file, the following types of zip code are found:
- 02446
- 02445-5841
- MA 02116
- Unknown

All these zip code types have at least 5 digits in their string. The following code will extract the first 5 digits from zip code string and then checks to see if the extracted zip code is located in Boston area or not.

All the zip codes outside of Boston area and all the Unknown zip codes will be assigned as zero. I have provided a simple test so that you see what exactly is expected.


In [121]:
# A regular expression to find zipcodes (first five digits) within a string 
zipcode_re = re.compile(r'\d+')

# This function update zipcode using extracted digits from regular expression
# It returns "0" if the zipcode was not found or if it is outside of Boston area.
def update_zipcode(zipcode):
    zipcode = zipcode_re.findall(zipcode)
    
    if zipcode != [] and len(zipcode[0]) == 5:
        zipcode = zipcode[0]
        if int(zipcode) <= 1431 or int(zipcode) >= 2770:
            zipcode = '0'
    else:
        zipcode = '0'
    return zipcode

# Print out the examples of update function
for zipcode_type, num in zipcode_types.items()[0:5]:
    better_zipcode = update_zipcode(zipcode_type)
    print zipcode_type, "=>", better_zipcode
    

0239 => 0
02186 => 02186
02184 => 02184
02134-1327 => 02134
02189 => 02189


## Preparing CSV files for SQL Database

After auditing is complete the next step is to prepare the data to be inserted into a SQL database. To do so I will parse the elements in the OSM XML file, transforming them from document format to tabular format, thus making it possible to write to .csv files.  These csv files can then easily be imported to a SQL database as tables.

The process for this transformation is as follows:
- Use iterparse to iteratively step through each top level element in the XML
- Shape each element into several data structures using a custom function
- Utilize a schema and validation library to ensure the transformed data is in the correct format
- Write each data structure to the appropriate .csv files

The shape_element function will transform each element into the correct format. I have used the Udacity pre-defined schema (**schema.py** file) for the .csv files and the eventual tables. And finally, using the **cerberus** library I validated the output against this schema to ensure it is correct.

### If the element top level tag is "node":
The dictionary returned should have the format {"node": .., "node_tags": ...}

The "node" field should hold a dictionary of the following top level node attributes:
- id
- user
- uid
- version
- lat
- lon
- timestamp
- changeset
All other attributes can be ignored

The "node_tags" field should hold a list of dictionaries, one per secondary tag. Secondary tags are
child tags of node which have the tag name/type: "tag". Each dictionary should have the following
fields from the secondary tag attributes:
- id: the top level node id attribute value
- key: the full tag "k" attribute value if no colon is present or the characters after the colon if one is.
- value: the tag "v" attribute value
- type: either the characters before the colon in the tag "k" value or "regular" if a colon is not present.

Additionally,

- if the tag "k" value contains problematic characters, the tag should be ignored
- if the tag "k" value contains a ":" the characters before the ":" should be set as the tag type
  and characters after the ":" should be set as the tag key
- if there are additional ":" in the "k" value they and they should be ignored and kept as part of
  the tag key. 
- If a node has no secondary tags then the "node_tags" field should just contain an empty list.


### If the element top level tag is "way":
The dictionary should have the format {"way": ..., "way_tags": ..., "way_nodes": ...}

The "way" field should hold a dictionary of the following top level way attributes:
- id
-  user
- uid
- version
- timestamp
- changeset

All other attributes can be ignored

The "way_tags" field should again hold a list of dictionaries, following the exact same rules as
for "node_tags".

Additionally, the dictionary should have a field "way_nodes". "way_nodes" should hold a list of
dictionaries, one for each nd child tag.  Each dictionary should have the fields:
- id: the top level element (way) id
- node_id: the ref attribute value of the nd tag
- position: the index starting at 0 of the nd tag i.e. what order the nd tag appears within the way element


In [ ]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

#OSM_PATH = "boston_massachusetts_sample.osm"
OSM_PATH = "boston_massachusetts.osm"

# Pathes to save the CSV files
NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

# Regular expression to find tags with a colon in their names (lower_colon)
# or tags with problematic characters (problemchars).    
LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

# The Udacity pre-defined schema to transform each element into the correct format. 
SCHEMA = schema.schema

# the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

# The shape_element function will transform each element into the correct format. 
# using schema.py file and checks the format using the cerberus library 
# and their respective values using update functions.
def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        node_attribs['id'] = element.attrib['id']
        node_attribs['user'] = element.attrib['user']
        node_attribs['uid'] = element.attrib['uid']
        node_attribs['version'] = element.attrib['version']
        node_attribs['lat'] = element.attrib['lat']
        node_attribs['lon'] = element.attrib['lon']
        node_attribs['timestamp'] = element.attrib['timestamp']
        node_attribs['changeset'] = element.attrib['changeset']

        for tag in element.iter("tag"):
            d={}
            d['id'] = node_attribs['id']
            k = tag.attrib['k']
            if PROBLEMCHARS.match(k) == None:
                if LOWER_COLON.match(k) != None:
                    d['type'] = k.split(':')[0]
                    d['key'] = ':'.join(k.split(':')[1:])
                else:
                    d['type'] = 'regular'
                    d['key'] = k
                    
            if  k == "addr:street":
                d['value'] = update_street(tag.attrib['v'], mapping_street)
            elif  k == "addr:state":
                d['value'] = update_state(tag.attrib['v'], mapping_state)
            elif  k == "addr:postcode":
                d['value'] = update_zipcode(tag.attrib['v'])
            else:
                d['value'] = tag.attrib['v']
            tags.append(d)
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        way_attribs['id'] = element.attrib['id']
        way_attribs['user'] = element.attrib['user']
        way_attribs['uid'] = element.attrib['uid']
        way_attribs['version'] = element.attrib['version']
        way_attribs['timestamp'] = element.attrib['timestamp']
        way_attribs['changeset'] = element.attrib['changeset']

        for tag in element.iter("tag"):
            d={}
            d['id'] = way_attribs['id']
            k = tag.attrib['k']
            if PROBLEMCHARS.match(k) == None:
                if LOWER_COLON.match(k) != None:
                    d['type'] = k.split(':')[0]
                    d['key'] = ':'.join(k.split(':')[1:])
                else:
                    d['type'] = 'regular'
                    d['key'] = k
                    
            if  k == "addr:street":
                d['value'] = update_street(tag.attrib['v'], mapping_street)
            elif  k == "addr:state":
                d['value'] = update_state(tag.attrib['v'], mapping_state)
            elif  k == "addr:postcode":
                d['value'] = update_zipcode(tag.attrib['v'])
            else:
                d['value'] = tag.attrib['v']
            tags.append(d)
        
        index = 0
        for tag in element.iter("nd"):
            d={}
            d['id'] = way_attribs['id']
            d['node_id'] = tag.attrib['ref']
            d['position'] = index
            way_nodes.append(d)
            index +=1
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}

def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

process_map(OSM_PATH, validate=True)

## Create SQL DB from CSV files
### Table for nodes


In [126]:
import csv
import sqlite3

# Creates the SQL database
con = sqlite3.connect('boston_massachusetts.db')
con.text_factory = str
cur = con.cursor()

# Creates the "nodes" table in the database
#cur.execute('''DROP TABLE nodes;''')
cur.execute('''CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT);''') 

# Inserting values to the "nodes" table from CSV files
with open ('nodes.csv', 'rb') as table:
    dicts = csv.DictReader(table)
    to_db = ((i['id'], i['lat'],i['lon'],i['user'],i['uid'],i['version'],i['changeset'],i['timestamp']) for i in dicts) 
    cur.executemany("INSERT INTO nodes (id, lat, lon, user, uid, version, changeset, timestamp) VALUES (?,?,?,?,?,?,?,?);", to_db)
con.commit()
#QUERY = '''PRAGMA table_info(nodes)'''
QUERY = '''SELECT id, lat, lon, user, uid, version, changeset FROM nodes LIMIT 5;'''
rows = cur.execute(QUERY).fetchall()
#print(rows)
pprint.pprint(rows)

[(30730952, 42.3678097, -71.0218711, 'wambag', 326503, 2, 14335103),
 (30730953, 42.3677364, -71.0218568, 'wambag', 326503, 2, 14335103),
 (30730954, 42.3676084, -71.0218168, 'wambag', 326503, 2, 14335103),
 (30730955, 42.3675229, -71.0218486, 'wambag', 326503, 2, 14335103),
 (30730956, 42.3674548, -71.0218865, 'wambag', 326503, 2, 14335103)]


### Table for nodes_tags

In [130]:
# Creates the "nodes_tags" table in the database
#cur.execute('''DROP TABLE nodes_tags;''')
cur.execute('''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id));''') 

# Inserting values to the "nodes_tags" table from CSV files
with open ('nodes_tags.csv', 'rb') as table:
    dicts = csv.DictReader(table)
    to_db = ((i['id'], i['key'],i['value'],i['type']) for i in dicts)  
    cur.executemany("INSERT INTO nodes_tags (id, key,value,type) VALUES (?,?,?,?);", to_db)
con.commit()
#QUERY = '''PRAGMA table_info(nodes_tags)'''
QUERY = '''SELECT * FROM nodes_tags LIMIT 5;'''
rows = cur.execute(QUERY).fetchall()
#print(rows)
pprint.pprint(rows)

[(31419556, 'name', 'Firebrand Saints', 'regular'),
 (31419556, 'amenity', 'restaurant', 'regular'),
 (31419650,
  'url',
  'http://www.mbta.com/schedules_and_maps/subway/lines/stations/?stopId=12412',
  'regular'),
 (31419650, 'name', 'Kendall/MIT', 'regular'),
 (31419650, 'railway', 'subway_entrance', 'regular')]


### Table for ways


In [30]:
# Creates the "ways" table in the database
#cur.execute('''DROP TABLE ways;''')
cur.execute('''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT);''') 

# Inserting values to the "ways" table from CSV files
with open ('ways.csv', 'rb') as table:
    dicts = csv.DictReader(table)
    to_db = ((i['id'], i['user'],i['uid'],i['version'],i['changeset'],i['timestamp']) for i in dicts)  
    cur.executemany("INSERT INTO ways (id, user,uid,version,changeset,timestamp) VALUES (?,?,?,?,?,?);", to_db)
con.commit()
#QUERY = '''PRAGMA table_info(ways)'''
QUERY = '''SELECT * FROM ways LIMIT 5;'''
rows = cur.execute(QUERY).fetchall()
#print(rows)
pprint.pprint(rows)

[(4790718, 'JessAk71', 381909, '3', 6963203, '2011-01-14T00:06:42Z'),
 (4790735, 'claysmalley', 119881, '4', 9069918, '2011-08-19T22:12:46Z'),
 (4824114, 'effektz', 3983283, '20', 39519438, '2016-05-23T20:54:09Z'),
 (4824115, 'ezr2', 1733549, '13', 32296955, '2015-06-29T23:22:38Z'),
 (4824116, 'effektz', 3983283, '24', 39518613, '2016-05-23T20:16:38Z')]


### Table for ways_tags


In [31]:
# Creates the "ways_tags" table in the database
#cur.execute('''DROP TABLE ways_tags;''')
cur.execute('''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id));''') 

# Inserting values to the "ways_tags" table from CSV files
with open ('ways_tags.csv', 'rb') as table:
    dicts = csv.DictReader(table)
    to_db = ((i['id'], i['key'],i['value'],i['type']) for i in dicts)  
    cur.executemany("INSERT INTO ways_tags(id, key,value,type) VALUES (?,?,?,?);", to_db)
con.commit()
#QUERY = '''PRAGMA table_info(ways_tags)'''
QUERY = '''SELECT * FROM ways_tags LIMIT 5;'''
rows = cur.execute(QUERY).fetchall()
#print(rows)
pprint.pprint(rows)

[(4790718, 'foot', 'yes', 'regular'),
 (4790718, 'name', 'Skybridge to Hilton', 'regular'),
 (4790718, 'layer', '1', 'regular'),
 (4790718, 'bridge', 'yes', 'regular'),
 (4790718, 'highway', 'footway', 'regular')]


### Table for ways_nodes 


In [32]:
# Creates the "ways_nodes" table in the database
#cur.execute('''DROP TABLE ways_nodes;''')
cur.execute('''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id));''') 

# Inserting values to the "ways_nodes" table from CSV files
with open ('ways_nodes.csv', 'rb') as table:
    dicts = csv.DictReader(table)
    to_db = ((i['id'], i['node_id'],i['position']) for i in dicts)  
    cur.executemany("INSERT INTO ways_nodes(id, node_id,position) VALUES (?,?,?);", to_db)
con.commit()
#QUERY = '''PRAGMA table_info(ways_nodes)'''
QUERY = '''SELECT * FROM ways_nodes LIMIT 5;'''
rows = cur.execute(QUERY).fetchall()
#print(rows)
pprint.pprint(rows)

[(4790718, 30730967, 0),
 (4790718, 30730968, 1),
 (4790718, 325383304, 2),
 (4790735, 1404985339, 0),
 (4790735, 1404985374, 1)]


## Data Overview 
### File Sizes


In [35]:
# Creates a list of file sizes
import os
files_lst = ['nodes.csv', 'nodes_tags.csv', 'ways.csv', 'ways_tags.csv', 'ways_nodes.csv',
             'boston_massachusetts_sample.db', 'boston_massachusetts.osm']
for i in files_lst: 
    print "file {!r} is {!s} MB".format(i,round(os.path.getsize(i)/(1024*1024.0),1))

file 'nodes.csv' is 151.8 MB
file 'nodes_tags.csv' is 16.9 MB
file 'ways.csv' is 20.0 MB
file 'ways_tags.csv' is 21.5 MB
file 'ways_nodes.csv' is 52.3 MB
file 'boston_massachusetts_sample.db' is 2.3 MB
file 'boston_massachusetts.osm' is 414.2 MB


### Number of nodes 

In [144]:
# Queries the Number of nodes
QUERY=('''SELECT COUNT(*) FROM nodes;''')
rows = cur.execute(QUERY).fetchall()
print rows[0][0]

1931442


### Number of ways 

In [153]:
# Queries the Number of ways
QUERY=('''SELECT COUNT(*) FROM ways;''')
rows = cur.execute(QUERY).fetchall()
print rows[0][0]

309066


## Data Exploration
This section contains basic statistics about the dataset, the SQL queries used to gather them, and some additional ideas about the data in context.
### Top Postal Codes

In [140]:
# Queries the Top 10 Postal Codes by count
QUERY=('''SELECT tags.value, COUNT(*) as count 
        FROM (SELECT * FROM nodes_tags 
        UNION ALL 
        SELECT * FROM ways_tags) tags
        WHERE tags.key='postcode'
        GROUP BY tags.value
        ORDER BY count DESC LIMIT 10;''')
rows = cur.execute(QUERY).fetchall()
for k, v in rows: print k, ':', v

02139 : 431
02135 : 268
02130 : 182
02134 : 162
02474 : 133
02144 : 117
02138 : 97
02114 : 86
02143 : 67
02145 : 64


### Sort cities by count, descending

In [145]:
# Queries the Top 10 cities by count
QUERY=('''SELECT tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL 
      SELECT * FROM ways_tags) tags
WHERE tags.key LIKE '%city'
GROUP BY tags.value
ORDER BY count DESC
LIMIT 10;''')
rows = cur.execute(QUERY).fetchall()
for k, v in rows: print k, ':', v


Boston : 949
Cambridge : 556
Malden : 413
1 : 411
Arlington : 285
Somerville : 245
Jamaica Plain : 96
2 : 92
Quincy : 55
15 : 52


### Number of  unique users 

In [146]:
# Queries the Number of unique users
QUERY=('''SELECT COUNT(DISTINCT(e.uid))          
FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e;''')
rows = cur.execute(QUERY).fetchall()
print rows[0][0]

1295


### Top 10 contributing users

In [147]:
# Queries Top 10 contributing users
QUERY=('''SELECT e.user, COUNT(*) as num
FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
GROUP BY e.user
ORDER BY num DESC
LIMIT 10;''')
rows = cur.execute(QUERY).fetchall()
for k, v in rows: print k, ':', v


crschmidt : 1202965
jremillard-massgis : 430112
OceanVortex : 92067
wambag : 80064
morganwahl : 69535
ryebread : 67063
MassGIS Import : 63277
ingalls_imports : 32461
Ahlzen : 27154
mapper999 : 14967


### Number of users appearing only once (having 1 post)

In [148]:
# Queries the Number of users appearing only once (having 1 post)
con = sqlite3.connect('boston_massachusetts.db')
con.text_factory = str
cur = con.cursor()
QUERY=('''SELECT COUNT(*) 
FROM
    (SELECT e.user, COUNT(*) as num
     FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
     GROUP BY e.user
     HAVING num=1)  u;''')
rows = cur.execute(QUERY).fetchall()
print rows[0][0]

355


### Top 10 appearing amenities

In [149]:
# Queries Top 10 appearing amenities
QUERY=('''SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
ORDER BY num DESC
LIMIT 10;''')
rows = cur.execute(QUERY).fetchall()
for k, v in rows: print k, ':', v


bench : 1060
restaurant : 602
school : 509
place_of_worship : 287
library : 280
bicycle_parking : 273
cafe : 248
fast_food : 184
bicycle_rental : 138
post_box : 114


### Biggest religion (no surprise here)



In [152]:
# Queries the Biggest religion
QUERY=('''SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='place_of_worship') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='religion'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 1;''')
rows = cur.execute(QUERY).fetchall()
for k, v in rows: print k, ':', v

christian : 255


### Most popular cuisines



In [154]:
# Queries Top 10 popular cuisines
QUERY=('''SELECT nodes_tags.value, COUNT(*) as num
FROM nodes_tags 
    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
    ON nodes_tags.id=i.id
WHERE nodes_tags.key='cuisine'
GROUP BY nodes_tags.value
ORDER BY num DESC
LIMIT 10;''')
rows = cur.execute(QUERY).fetchall()
for k, v in rows: print k, ':', v

pizza : 38
american : 35
italian : 31
chinese : 29
mexican : 27
indian : 21
thai : 19
asian : 13
japanese : 12
regional : 12


## Additional Ideas
One of the most useful attribute of all tags is the "building" key. People are looking up and searching for different types of buildings very often and on a daily basis. Having the OSM database updated with such an information is necessary. As of now, everybody's using "Yelp" or "Google maps" since they have a vast data on the type of buildings. 

Looking at Boston OSM file, I found out that only a handful of buildings (less than 400) have assigned a value for the "building" key where most of them (334) have assigned "yes" as the value which is incorrect and useless.

As a suggestion, I think OpenStreetMap.org can implement a structure that ask users and refer them to a pre-defined table for choosing the type of buildings as an input.
Moreover, this table can provide different examples for each type of buildings and helps users to choose more carefully and create more accurate database. 

The benefits of such an improvement in database is obvious (as I explained above) but it's not an easy process and some problems can occur such as:
- If we use a complicated table, users may prefer to skip inputting such an information, or use a random inaccurate inputs like the case of inputting 'yes'. 
- Most of the buildings have multiple usage and can be assigned with different key values. This again can confuse and frustrate the users.

I think we should look for some rewards or motivational procedures so users spend some time to update the database in this way.


In [155]:
# Queries all types of buildings
QUERY=('''SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='building'
GROUP BY value
ORDER BY num DESC;''')
rows = cur.execute(QUERY).fetchall()
for k, v in rows: print k, ':', v

yes : 334
entrance : 29
commercial : 7
school : 6
retail : 3
apartments : 2
office : 2
residential : 2
university : 2
brewery : 1
chapel : 1
church : 1
dormitory : 1
industrial : 1
lot : 1
public : 1
warehouse : 1


## Conclusion
In this project, we tried to audit and clean the most popular tags (**street**, **state** and **postcode**) in **Boston OSM file** and obviously lots of cleaning processes are remained which are out of scope of this project. However, the significant number of users and their contributions to OpenStreetMap.org is very promising and I think we will have most of the OSM data cleaned in the near future. 